In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../../q1_o2kr2_dataset_annotations/')

from collections import defaultdict
import json
import os
import cv2
from matplotlib import pyplot as plt
import matplotlib.patches as patches
import numpy as np
import pandas as pd
from research_lib.utils.data_access_utils import S3AccessUtils
from research.utils.data_access_utils import RDSAccessUtils
import uuid
from construct_fish_detection_dataset_o2kr2 import establish_plali_connection, insert_into_plali
from rectification import rectify
from weight_estimation.weight_estimator import WeightEstimator, CameraMetadata
from research.weight_estimation.keypoint_utils.optics import pixel2world

from PIL import Image, ImageDraw, ImageFont


<h1> Get raw images from toy fish experiment </h1>

In [ ]:
s3 = S3AccessUtils('/root/data')

In [ ]:
prefix = 'environment=production/site-id=55/pen-id=102/hour=11'
suffixes = ['frame.jpg']
keygen = s3.get_matching_s3_keys('aquabyte-images-raw', prefix, suffixes=suffixes)
keys = []
for key in keygen:
    keys.append(key)


In [ ]:
image_pair_dict = defaultdict(dict)
for key in keys:
    dirname = os.path.dirname(key)
    if 'left' in key:
        image_pair_dict[dirname]['left'] = key
    elif 'right' in key:
        image_pair_dict[dirname]['right'] = key
    else:
        raise Exception('Key not valid')
        
image_pairs = []
for dirname in sorted(list(image_pair_dict.keys())):
    keys = image_pair_dict[dirname]
    try:
        image_pairs.append((keys['left'], keys['right']))
    except KeyError as err:
        print(err)
        
    

<h1> Check frames </h1>

In [ ]:
def download_image(image_url):
    image_s3_url = image_url
    url_components = image_s3_url.replace('s3://', '').split('/')
    bucket = url_components[0]
    key = os.path.join(*url_components[1:])
    image_f = s3.download_from_s3(bucket, key)
    return image_f


def plot_stereo_image(left_image_f, right_image_f):
    fig, axes = plt.subplots(1, 2)
    
    # show left image
    left_im = cv2.imread(left_image_f)
    left_im = cv2.cvtColor(left_im, cv2.COLOR_BGR2RGB)
    axes[0].imshow(left_im)
    
    # show right image
    right_im = cv2.imread(right_image_f)
    right_im = cv2.cvtColor(right_im, cv2.COLOR_BGR2RGB)
    axes[1].imshow(right_im)
    
    plt.show()
    


In [ ]:
idx = 0
for left_key, right_key in image_pairs:
    print(idx)
    idx += 1
    
    left_full_res_frame_s3_url, right_full_res_frame_s3_url = [os.path.join('s3://', 'aquabyte-images-raw', key) for key in (left_key, right_key)]
    left_frame_s3_url, right_frame_s3_url = [x.replace('.jpg', '.resize_512_512.jpg') for x in (left_full_res_frame_s3_url, right_full_res_frame_s3_url)]
    
    # download left image
    left_image_f = download_image(left_frame_s3_url)
    right_image_f = download_image(right_frame_s3_url)
    
    # plot image
    plot_stereo_image(left_image_f, right_image_f)
    

<h1> Rectify raw images and upload to s3 </h1>

In [ ]:
def download_from_s3_url(s3_url):
    url_components = s3_url.replace('s3://', '').split('/')
    bucket = url_components[0]
    key = os.path.join(*url_components[1:])
    f = s3.download_from_s3(bucket, key)
    return f, bucket, key

In [ ]:
left_image_rectified_s3_urls, right_image_rectified_s3_urls = [], []
stereo_parameters_url = 'https://aquabyte-abc.s3-eu-west-1.amazonaws.com/rook/2021-03-10T13%3A57%3A48Z-pfe-1421920048928-187-4bd8/cal_output/2021-03-10T14-07-03.821272000Z/stereo_params.json'.replace('%3A', ':')
count = 0

for left_key, right_key in image_pairs:
    
    # get unrectified full resolution frames
    left_full_res_frame_s3_url, right_full_res_frame_s3_url = [os.path.join('s3://', 'aquabyte-images-raw', key) for key in (left_key, right_key)]
    left_full_res_frame_f, _, left_full_res_frame_key = download_from_s3_url(left_full_res_frame_s3_url)
    right_full_res_frame_f, _, right_full_res_frame_key = download_from_s3_url(right_full_res_frame_s3_url)
    stereo_parameters_f, _, _ = s3.download_from_url(stereo_parameters_url)
    
    # rectify into full resolution stereo frame pair and save to disk
    left_image_rectified, right_image_rectified = rectify(left_full_res_frame_f, right_full_res_frame_f, stereo_parameters_f)
    left_image_rectified_f = os.path.join(os.path.dirname(left_full_res_frame_f), 'left_frame.rectified.jpg')
    right_image_rectified_f = os.path.join(os.path.dirname(right_full_res_frame_f), 'right_frame.rectified.jpg')
    cv2.imwrite(left_image_rectified_f, left_image_rectified)
    cv2.imwrite(right_image_rectified_f, right_image_rectified)
    
    # upload rectified stereo frame pairs to s3
    left_rectified_full_res_frame_key = left_full_res_frame_key.replace('.jpg', '.rectified.jpg')
    right_rectified_full_res_frame_key = right_full_res_frame_key.replace('.jpg', '.rectified.jpg')
    s3.s3_client.upload_file(left_image_rectified_f, 'aquabyte-images-raw', left_rectified_full_res_frame_key)
    s3.s3_client.upload_file(right_image_rectified_f, 'aquabyte-images-raw', right_rectified_full_res_frame_key)
    
    # append to url lists
    left_image_rectified_s3_url = os.path.join('s3://', 'aquabyte-images-raw', left_rectified_full_res_frame_key)
    right_image_rectified_s3_url = os.path.join('s3://', 'aquabyte-images-raw', right_rectified_full_res_frame_key)
    left_image_rectified_s3_urls.append(left_image_rectified_s3_url)
    right_image_rectified_s3_urls.append(right_image_rectified_s3_url)
    
    print(count)
    count += 1




<h1> Insert into PLALI for key-point annotation </h1>

In [ ]:
def process_into_plali_records(image_url_pairs, workflow_id):

    values_to_insert = []
    for idx, image_url_pair in enumerate(image_url_pairs):
        id = str(uuid.uuid4())
        images = set(image_url_pair)
        metadata = { 'type': 'Dale P3 post-swap enclosure' }
        priority = float(idx) / len(image_url_pairs)

        values = {
            'id': id,
            'workflow_id': workflow_id,
            'images': images,
            'metadata': metadata,
            'priority': priority
        }

        values_to_insert.append(values)

    return values_to_insert


def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))



In [ ]:
image_url_pairs = list(zip(left_image_rectified_s3_urls, right_image_rectified_s3_urls))

In [ ]:
WORKFLOW_ID = '00000000-0000-0000-0000-000000000055'
values_to_insert = process_into_plali_records(image_url_pairs, WORKFLOW_ID)



In [ ]:
os.environ['PLALI_SQL_CREDENTIALS'] = '/run/secrets/plali_sql_credentials'

engine, sql_metadata = establish_plali_connection()

n = 10
count = 0
for chunk in chunker(values_to_insert[:], n):
    insert_into_plali(chunk, engine, sql_metadata)
    
    count += 1
    print(count)

In [ ]:
annotated_df.images.iloc[0]

<h1> Calculate weights </h1>

<h2> Parse annotations into standard form </h2>

In [ ]:
os.environ['PLALI_SQL_CREDENTIALS'] = '/run/secrets/plali_sql_credentials'
rds = RDSAccessUtils(json.load(open(os.environ['PLALI_SQL_CREDENTIALS'])))

query = """
    select * from plali.plali_annotations x
    inner join 
    ( select a.id as plali_image_id, a.images, a.metadata, b.id as workflow_id, b.name from plali.plali_images a
    inner join plali.plali_workflows b
    on a.workflow_id = b.id ) y
    on x.plali_image_id = y.plali_image_id
    where workflow_id = '00000000-0000-0000-0000-000000000055';
"""

annotated_df = rds.extract_from_database(query)
annotated_df = annotated_df[annotated_df.metadata.apply(lambda x: x.get('type') is not None)]

In [ ]:
annotated_df.metadata

In [ ]:
annotated_df.weight.median()

In [ ]:
class AnnotationFormatError(Exception):
    pass


anns = []
for idx, row in annotated_df.iterrows():
    try:
        raw_ann = row.annotation
        if 'skipReasons' in raw_ann:
            raise AnnotationFormatError

        ann = {'leftCrop': [], 'rightCrop': []}

        for side in ['leftCrop', 'rightCrop']:
            for raw_item in row.annotation[side]['annotation']['annotations']:
                if 'xCrop' not in raw_item or 'yCrop' not in raw_item:
                    raise AnnotationFormatError
                item = {
                    'xCrop': raw_item['xCrop'],
                    'yCrop': raw_item['yCrop'],
                    'xFrame': raw_item['xCrop'],
                    'yFrame': raw_item['yCrop'],
                    'keypointType': raw_item['category']
                }
                
                ann[side].append(item)

        if any([len(ann[side]) != 11 for side in ['leftCrop', 'rightCrop']]):
            raise AnnotationFormatError
        
        anns.append(ann)
        
    except AnnotationFormatError as err:
        anns.append(None)
    
    
    

In [ ]:
annotated_df['ann'] = anns

<h2> Check annotations / disparity values </h2>

In [ ]:
for idx, row in annotated_df.iterrows():
    ann = row.ann
    if ann is not None:
        left_mean_x = np.mean([item['xFrame'] for item in ann['leftCrop']])
        right_mean_x = np.mean([item['xFrame'] for item in ann['rightCrop']])
        print(left_mean_x - right_mean_x)


<h2> Compute weights </h2>

In [ ]:
stereo_parameters_url = 'https://aquabyte-abc.s3-eu-west-1.amazonaws.com/rook/2021-03-10T13%3A57%3A48Z-pfe-1421920048928-187-4bd8/cal_output/2021-03-10T14-07-03.821272000Z/stereo_params.json'.replace('%3A', ':')
stereo_parameters_f, _, _ = s3.download_from_url(stereo_parameters_url)

stereo_params = json.load(open(stereo_parameters_f))
camera_metadata = {
    'focalLengthPixel': stereo_params['CameraParameters1']['FocalLength'][0],
    'baseline': abs(stereo_params['TranslationOfCamera2'][0] / 1e3),
    'focalLength': stereo_params['CameraParameters1']['FocalLength'][0] * 3.45e-6,
    'pixelCountWidth': 4096,
    'pixelCountHeight': 3000,
    'imageSensorWidth': 0.01412,
    'imageSensorHeight': 0.01035
}

In [ ]:
weight_model_f, _, _ = s3.download_from_url('https://aquabyte-models.s3-us-west-1.amazonaws.com/biomass/trained_models/2020-11-27T00-00-00/weight_model_synthetic_data.pb')
kf_model_f, _, _ = s3.download_from_url('https://aquabyte-models.s3-us-west-1.amazonaws.com/k-factor/trained_models/2020-08-08T000000/kf_predictor_v2.pb')
weight_estimator = WeightEstimator(weight_model_f, kf_model_f)

pred_weights, pred_kfs = [], []

count = 0
for idx, row in annotated_df.iterrows():
    ann = row.ann
    if ann is not None:
        cm = CameraMetadata(
            focal_length=camera_metadata['focalLength'],
            focal_length_pixel=camera_metadata['focalLengthPixel'],
            baseline_m=camera_metadata['baseline'],
            pixel_count_width=camera_metadata['pixelCountWidth'],
            pixel_count_height=camera_metadata['pixelCountHeight'],
            image_sensor_width=camera_metadata['imageSensorWidth'],
            image_sensor_height=camera_metadata['imageSensorHeight']
        )

        weight, _, kf = weight_estimator.predict(ann, cm)
        pred_weights.append(weight)
        pred_kfs.append(kf)
    else:
        pred_weights.append(None)
        pred_kfs.append(None)
    
    if count % 1000 == 0:
        print(count)
    count += 1
    

In [ ]:
annotated_df['weight'] = pred_weights
annotated_df['kf'] = pred_kfs

In [ ]:
annotated_df['weight'].median()

In [ ]:
annotated_df['kf'].mean()

<h1> Calculate length </h1>

In [ ]:
from research.weight_estimation.keypoint_utils.optics import pixel2world

def calculate_length_1(ann, cm):
    if ann is None:
        return None

    wkps = pixel2world(ann['leftCrop'], ann['rightCrop'], cm)
    return np.linalg.norm(wkps['UPPER_LIP'] - wkps['TAIL_NOTCH'])

def calculate_length_2(ann, cm):
    if ann is None:
        return None

    wkps = pixel2world(ann['leftCrop'], ann['rightCrop'], cm)
    return np.linalg.norm(wkps['DORSAL_FIN'] - wkps['PELVIC_FIN'])

def calculate_length_3(ann, cm):
    if ann is None:
        return None

    wkps = pixel2world(ann['leftCrop'], ann['rightCrop'], cm)
    return np.linalg.norm(wkps['ADIPOSE_FIN'] - wkps['PELVIC_FIN'])

In [ ]:
annotated_df['length_1'] = annotated_df.ann.apply(lambda x: calculate_length_1(x, camera_metadata))
annotated_df['length_2'] = annotated_df.ann.apply(lambda x: calculate_length_2(x, camera_metadata))
annotated_df['length_3'] = annotated_df.ann.apply(lambda x: calculate_length_3(x, camera_metadata))

In [ ]:
print(annotated_df.length_1.median())
print(annotated_df.length_2.median())
print(annotated_df.length_3.median())

In [ ]:
plt.figure(figsize=(10, 5))
plt.hist(annotated_df['length_3'], bins=50)
plt.xlim([0.06, 0.1])
plt.grid()
plt.show()

In [ ]:
def calculate_depth(ann, cm):
    if ann is None:
        return None

    wkps = pixel2world(ann['leftCrop'], ann['rightCrop'], cm)
    return np.median([x[1] for x in wkps.values()])

In [ ]:
annotated_df['length_1'] = annotated_df.ann.apply(lambda x: calculate_length_1(x, camera_metadata))
annotated_df['length_2'] = annotated_df.ann.apply(lambda x: calculate_length_2(x, camera_metadata))

In [ ]:
plt.figure(figsize=(10, 5))
plt.hist(annotated_df['length_3'], bins=200)
plt.xlim(0.12, 0.15)
plt.grid()
plt.show()

In [ ]:
annotated_df['length_2'].median()

In [ ]:
plt.figure(figsize=(20, 10))
plt.xlim([0.125, 0.2])
plt.hist(annotated_df['length_2'], bins=200)
plt.grid()
plt.show()

In [ ]:
camera_metadata

In [ ]:
annotated_df['length'] = annotated_df.ann.apply(lambda x: calculate_length(x, camera_metadata))

In [ ]:
annotated_df['depth'] = annotated_df.ann.apply(lambda x: calculate_depth(x, camera_metadata))

In [ ]:
plt.figure(figsize=(15, 8))
plt.hist(annotated_df.weight, bins=20)
plt.axvline(annotated_df.weight.median(), color='red', linestyle='--')
plt.grid()
plt.show()

In [ ]:
plt.hist(annotated_df.length, bins=20)
plt.show()

In [ ]:
annotated_df.length.median()

In [ ]:
for w in annotated_df.weight:
    print(w)

<h1> Unrectify and Re-rectify </h1>

In [ ]:
from typing import Tuple

def load_params(params):
    print("Loading params...")
    cameraMatrix1 = np.array(params['CameraParameters1']['IntrinsicMatrix']).transpose()
    cameraMatrix2 = np.array(params['CameraParameters2']['IntrinsicMatrix']).transpose()

    distCoeffs1 = params['CameraParameters1']['RadialDistortion'][0:2] + \
                   params['CameraParameters1']['TangentialDistortion'] + \
                   [params['CameraParameters1']['RadialDistortion'][2]]
    distCoeffs1 = np.array(distCoeffs1)

    distCoeffs2 = params['CameraParameters2']['RadialDistortion'][0:2] + \
                   params['CameraParameters2']['TangentialDistortion'] + \
                   [params['CameraParameters2']['RadialDistortion'][2]]
    distCoeffs2 = np.array(distCoeffs2)

    R = np.array(params['RotationOfCamera2']).transpose()
    T = np.array(params['TranslationOfCamera2']).transpose()

    imageSize = (4096, 3000)

    # perform rectification
    (R1, R2, P1, P2, Q, leftROI, rightROI) = cv2.stereoRectify(cameraMatrix1, distCoeffs1, cameraMatrix2, distCoeffs2, imageSize, R, T, None, None, None, None, None, cv2.CALIB_ZERO_DISPARITY, 0)

    left_maps = cv2.initUndistortRectifyMap(cameraMatrix1, distCoeffs1, R1, P1, imageSize, cv2.CV_16SC2)
    right_maps = cv2.initUndistortRectifyMap(cameraMatrix2, distCoeffs2, R2, P2, imageSize, cv2.CV_16SC2)

    print("Params loaded.")
    return left_maps, right_maps

IMAGE_WIDTH = 4096
IMAGE_HEIGHT = 3000

def get_camera_parameters(params: dict) -> Tuple:
    """Return individual camera parameters from JSON stereo parameters contents."""
    
    cameraMatrix1 = np.array(params['CameraParameters1']['IntrinsicMatrix']).transpose()
    cameraMatrix2 = np.array(params['CameraParameters2']['IntrinsicMatrix']).transpose()

    distCoeffs1 = params['CameraParameters1']['RadialDistortion'][0:2] + \
                   params['CameraParameters1']['TangentialDistortion'] + \
                   [params['CameraParameters1']['RadialDistortion'][2]]
    distCoeffs1 = np.array(distCoeffs1)

    distCoeffs2 = params['CameraParameters2']['RadialDistortion'][0:2] + \
                   params['CameraParameters2']['TangentialDistortion'] + \
                   [params['CameraParameters2']['RadialDistortion'][2]]
    distCoeffs2 = np.array(distCoeffs2)

    R = np.array(params['RotationOfCamera2']).transpose()
    T = np.array(params['TranslationOfCamera2']).transpose()
    
    imageSize = (IMAGE_WIDTH, IMAGE_HEIGHT)
    (R1, R2, P1, P2, Q, leftROI, rightROI) = cv2.stereoRectify(cameraMatrix1, distCoeffs1, cameraMatrix2, 
                                                               distCoeffs2, imageSize, R, T, None, None, 
                                                               None, None, None, cv2.CALIB_ZERO_DISPARITY, 0)
    left_maps = cv2.initUndistortRectifyMap(cameraMatrix1, distCoeffs1, R1, P1, imageSize, cv2.CV_16SC2)
    right_maps = cv2.initUndistortRectifyMap(cameraMatrix2, distCoeffs2, R2, P2, imageSize, cv2.CV_16SC2)
    
    return left_maps, right_maps, cameraMatrix1, distCoeffs1, R1, P1, cameraMatrix2, distCoeffs2, R2, P2

In [ ]:
stereo_parameters_o_url = 'https://aquabyte-stereo-parameters.s3-eu-west-1.amazonaws.com/L40020184_R40029797/2021-02-25T11%3A54%3A31.569694000Z_L40020184_R40029797_stereo-parameters.json'.replace('%3A', ':')
stereo_parameters_n_url = 'https://aquabyte-stereo-parameters.s3-eu-west-1.amazonaws.com/L40029797_R40020184/2021-02-25T11%3A30%3A42.149694000Z_L40029797_R40020184_stereo-parameters.json'.replace('%3A', ':')

stereo_parameters_o_f, _, _ = s3.download_from_url(stereo_parameters_o_url)
stereo_parameters_n_f, _, _ = s3.download_from_url(stereo_parameters_n_url)

stereo_params_o = json.load(open(stereo_parameters_o_f))
stereo_params_n = json.load(open(stereo_parameters_n_f))
left_maps_o, right_maps_o, cameraMatrix1_o, distCoeffs1_o, R1_o, P1_o, cameraMatrix2_o, distCoeffs2_o, R2_o, P2_o = get_camera_parameters(stereo_params_o)
left_maps_n, right_maps_n, cameraMatrix1_n, distCoeffs1_n, R1_n, P1_n, cameraMatrix2_n, distCoeffs2_n, R2_n, P2_n = get_camera_parameters(stereo_params_n)

ann_u_rs = []
for idx, row in annotated_df.iterrows():
    ann = row.ann
    if ann is None:
        ann_u_rs.append(None)
        continue
    
    # un-rectify with matlab params, re-rectify with circular params
    ann_u_r = {'leftCrop': [], 'rightCrop': []}
    for side in ['leftCrop', 'rightCrop']:
        for item in ann[side]:
            bp = item['keypointType']
            x = item['xFrame']
            y = item['yFrame']
            if side == 'leftCrop':
                x_new, y_new = cv2.undistortPoints(np.array([[left_maps_o[0][y, x]]]).astype(float), 
                                    cameraMatrix1_n, distCoeffs1_n, R=R1_n, P=P1_n)[0][0]
            elif side == 'rightCrop':
                x_new, y_new = cv2.undistortPoints(np.array([[right_maps_o[0][y, x]]]).astype(float), 
                                    cameraMatrix2_n, distCoeffs2_n, R=R2_n, P=P2_n)[0][0]
                
            x_new, y_new = int(round(x_new)), int(round(y_new))
            ann_u_r[side].append({
                'keypointType': bp,
                'xFrame': x_new,
                'yFrame': y_new,
            })
    
    ann_u_rs.append(ann_u_r)

annotated_df['ann_u_r'] = ann_u_rs

In [ ]:
stereo_parameters_url = 'https://aquabyte-stereo-parameters.s3-eu-west-1.amazonaws.com/L40020184_R40029797/2021-02-19T13:20:20.603005000Z_L40020184_R40029797_stereo-parameters.json'
stereo_parameters_f, _, _ = s3.download_from_url(stereo_parameters_url)

stereo_params = json.load(open(stereo_parameters_f))
camera_metadata = {
    'focalLengthPixel': stereo_params['CameraParameters1']['FocalLength'][0],
    'baseline': abs(stereo_params['TranslationOfCamera2'][0] / 1e3),
    'focalLength': stereo_params['CameraParameters1']['FocalLength'][0] * 3.45e-6,
    'pixelCountWidth': 4096,
    'pixelCountHeight': 3000,
    'imageSensorWidth': 0.01412,
    'imageSensorHeight': 0.01035
}

In [ ]:
weight_model_f, _, _ = s3.download_from_url('https://aquabyte-models.s3-us-west-1.amazonaws.com/biomass/trained_models/2020-11-27T00-00-00/weight_model_synthetic_data.pb')
kf_model_f, _, _ = s3.download_from_url('https://aquabyte-models.s3-us-west-1.amazonaws.com/k-factor/trained_models/2020-08-08T000000/kf_predictor_v2.pb')
weight_estimator = WeightEstimator(weight_model_f, kf_model_f)

pred_weights = []

count = 0
for idx, row in annotated_df.iterrows():
    ann = row.ann_u_r
    if ann is not None:
        cm = CameraMetadata(
            focal_length=camera_metadata['focalLength'],
            focal_length_pixel=camera_metadata['focalLengthPixel'],
            baseline_m=camera_metadata['baseline'],
            pixel_count_width=camera_metadata['pixelCountWidth'],
            pixel_count_height=camera_metadata['pixelCountHeight'],
            image_sensor_width=camera_metadata['imageSensorWidth'],
            image_sensor_height=camera_metadata['imageSensorHeight']
        )

        weight, _, _ = weight_estimator.predict(ann, cm)
        pred_weights.append(weight)
    else:
        pred_weights.append(None)
    
    if count % 1000 == 0:
        print(count)
    count += 1
    

In [ ]:
annotated_df['weight_u_r'] = pred_weights

In [ ]:
annotated_df.weight.mean()

In [ ]:
annotated_df.weight_u_r.mean()

In [ ]:
plt.figure(figsize=(15, 8))
plt.hist(annotated_df.weight_u_r.values, bins=20)
plt.grid()
plt.show()

In [ ]:
def display_crops(left_image_f, right_image_f, ann, overlay_keypoints=True, show_labels=True):

    fig, axes = plt.subplots(2, 1, figsize=(20, 20))
    left_image = plt.imread(left_image_f)
    right_image = plt.imread(right_image_f)
    axes[0].imshow(left_image)
    axes[1].imshow(right_image)
    
    left_keypoints = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in ann['leftCrop']}
    right_keypoints = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in ann['rightCrop']}
    
    if overlay_keypoints:
        for bp, kp in left_keypoints.items():
            axes[0].scatter([kp[0]], [kp[1]], color='red', s=1)
            if show_labels:
                axes[0].annotate(bp, (kp[0], kp[1]), color='red')
        for bp, kp in right_keypoints.items():
            axes[1].scatter([kp[0]], [kp[1]], color='red', s=1)
            if show_labels:
                axes[1].annotate(bp, (kp[0], kp[1]), color='red')
    plt.show()

In [ ]:
count = 0
for idx, row in annotated_df.iterrows():
    ann = row.ann
    if ann is None:
        continue
    
    left_image_s3_url = row.images[0]
    right_image_s3_url = row.images[1]
    left_image_key = os.path.join(*left_image_s3_url.replace('s3://', '').split('/')[1:])
    right_image_key = os.path.join(*right_image_s3_url.replace('s3://', '').split('/')[1:])
    left_image_key, right_image_key = sorted([left_image_key, right_image_key])
    left_image_f = s3.download_from_s3('aquabyte-images-raw', left_image_key)
    right_image_f = s3.download_from_s3('aquabyte-images-raw', right_image_key)
    
    print(row.length_2)
    display_crops(left_image_f, right_image_f, ann)
    
#     if count > 10:
#         break
    count += 1

    

In [ ]:
def _add_depth(df):
    yaws, pitches, rolls, depths = [], [], [], []
    for idx, row in df.iterrows():
#         ann, cm = row.annotation, row.camera_metadata
        ann, cm = row.ann, camera_metadata
        try:
            world_keypoints = pixel2world(ann['leftCrop'], ann['rightCrop'], cm)
            depth = np.median([x[1] for x in world_keypoints.values()])
            u = world_keypoints['DORSAL_FIN'] - world_keypoints['PELVIC_FIN']
            v = world_keypoints['UPPER_LIP'] - world_keypoints['TAIL_NOTCH']
            yaw = np.arctan(v[1] / abs(v[0])) * (180.0 / np.pi)
            pitch = np.arctan(v[2] / abs(v[0])) * (180.0 / np.pi)
            roll = np.arctan(u[1] / u[0]) * (180.0 / np.pi)
        except TypeError as err:
            yaw, pitch, roll, depth = None, None, None, None
        yaws.append(yaw)
        pitches.append(pitch)
        depths.append(depth)
        rolls.append(roll)
        
    df['yaw'] = yaws
    df['pitch'] = pitches
    df['roll'] = rolls
    df['depth'] = depths
    return df

In [ ]:
plt.figure(figsize=(10, 5))
plt.scatter(annotated_df.yaw, annotated_df.weight)
plt.grid()
plt.show()


In [ ]:
annotated_df[annotated_df.roll.abs() < 10].weight.median()

In [ ]:
plt.figure(figsize=(10, 5))
plt.hist(annotated_df.pitch)
plt.grid()
plt.show()

In [ ]:
plt.scatter(annotated_df[mask].depth, annotated_df[mask].weight)

In [ ]:
plt.figure(figsize=(20, 10))
mask = annotated_df.length_2 < 0.2
plt.scatter(annotated_df[mask].length_2, annotated_df[mask].weight)
plt.grid()
plt.show()

<h1> Generate video </h1>

In [ ]:
S3_DIR = '/root/data/s3'
OUTPUT_BASE_DIR = '/root/data/alok/biomass_estimation/playground/toy_fish_video_dale_post_swap_enclosure'
WIDTH = 512

def stitch_frames(left_thumbnail_f, right_thumbnail_f, weight):

    # open images and metadata files
    left_im = Image.open(left_thumbnail_f)
    right_im = Image.open(right_thumbnail_f)

    # stitch images
    result = Image.new('RGB', (2 * WIDTH, WIDTH))
    result.paste(im=left_im, box=(0, 0))
    result.paste(im=right_im, box=(WIDTH, 0))

    # write timestamp on stitched image
    result_draw = ImageDraw.Draw(result)
#     selected_font = "arial.ttf"
#     font_size = 30
#     font = ImageFont.truetype(selected_font, font_size)
    result_draw.text((0, 0), '{} g'.format(str(weight)), (255, 255, 255))

    output_f = left_thumbnail_f.replace(S3_DIR, OUTPUT_BASE_DIR).replace('left_', 'stereo_')
    if not os.path.exists(os.path.dirname(output_f)):
        os.makedirs(os.path.dirname(output_f))
    result.save(output_f)
    return output_f
    

def stitch_frames_into_video(image_fs, video_f):
    im = cv2.imread(image_fs[0])
    height, width, layers = im.shape
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    video = cv2.VideoWriter(video_f, fourcc, 1, (width, height), True)
    for idx, image_f in enumerate(image_fs):
        if idx % 1000 == 0:
            print(idx)
        im = cv2.imread(image_f, cv2.IMREAD_COLOR)
        video.write(im)
    cv2.destroyAllWindows()
    video.release()

In [ ]:
mask = ~annotated_df.ann.isnull()
output_fs = []
for idx, row in annotated_df[mask].iterrows():
    left_image_s3_url = row.images[0]
    right_image_s3_url = row.images[1]
    weight = round(row.weight, 2)
    left_image_key = os.path.join(*left_image_s3_url.replace('s3://', '').split('/')[1:])
    right_image_key = os.path.join(*right_image_s3_url.replace('s3://', '').split('/')[1:])
    left_thumbnail_key, right_thumbnail_key = [x.replace('.rectified.jpg', '.resize_512_512.jpg') for x in (left_image_key, right_image_key)
                                              ]
    left_thumbnail_f = s3.download_from_s3('aquabyte-images-raw', left_thumbnail_key)
    right_thumbnail_f = s3.download_from_s3('aquabyte-images-raw', right_thumbnail_key)
    
    output_f = stitch_frames(left_thumbnail_f, right_thumbnail_f, weight)
    output_fs.append(output_f)
    

    

In [ ]:
[output_fs]

In [ ]:
stitch_frames_into_video(sorted(output_fs), '/root/data/alok/biomass_estimation/playground/toy_fish_video_dale_post_swap_enclosure/toy_fish_video_dale_post_swap_enclosure.avi')




In [ ]:
annotated_df.weight_u_r.median()